# 目的

- nb001で売り買いできることがわかったから戦略を考える

- bitfrayerの取引データをどこかから探してくる！（ろうそく足データ）

https://blog.shoby.jp/entry/2017/05/24/093733#f-e6c220ff

- 取引手数料

0.0000015　btｃ

In [34]:
import os 
import sys 
import time
import numpy as np
import pandas as pd
import talib

# 仮想通貨関連
import ccxt
import pybitflyer

# open high low close V

---

# richmanbtcさんの特徴量作成メソッド

In [39]:
def calc_features(df):
    open = df['op']
    high = df['hi']
    low = df['lo']
    close = df['cl']
    volume = df['volume']
    
    orig_columns = df.columns

    hilo = (df['hi'] + df['lo']) / 2
    df['BBANDS_upperband'], df['BBANDS_middleband'], df['BBANDS_lowerband'] = talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    df['BBANDS_upperband'] -= hilo
    df['BBANDS_middleband'] -= hilo
    df['BBANDS_lowerband'] -= hilo
    df['DEMA'] = talib.DEMA(close, timeperiod=30) - hilo
    df['EMA'] = talib.EMA(close, timeperiod=30) - hilo
    df['HT_TRENDLINE'] = talib.HT_TRENDLINE(close) - hilo
    df['KAMA'] = talib.KAMA(close, timeperiod=30) - hilo
    df['MA'] = talib.MA(close, timeperiod=30, matype=0) - hilo
    df['MIDPOINT'] = talib.MIDPOINT(close, timeperiod=14) - hilo
    df['SMA'] = talib.SMA(close, timeperiod=30) - hilo
    df['T3'] = talib.T3(close, timeperiod=5, vfactor=0) - hilo
    df['TEMA'] = talib.TEMA(close, timeperiod=30) - hilo
    df['TRIMA'] = talib.TRIMA(close, timeperiod=30) - hilo
    df['WMA'] = talib.WMA(close, timeperiod=30) - hilo

    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    df['ADXR'] = talib.ADXR(high, low, close, timeperiod=14)
    df['APO'] = talib.APO(close, fastperiod=12, slowperiod=26, matype=0)
    df['AROON_aroondown'], df['AROON_aroonup'] = talib.AROON(high, low, timeperiod=14)
    df['AROONOSC'] = talib.AROONOSC(high, low, timeperiod=14)
    df['BOP'] = talib.BOP(open, high, low, close)
    df['CCI'] = talib.CCI(high, low, close, timeperiod=14)
    df['DX'] = talib.DX(high, low, close, timeperiod=14)
    df['MACD_macd'], df['MACD_macdsignal'], df['MACD_macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    # skip MACDEXT MACDFIX たぶん同じなので
    df['MFI'] = talib.MFI(high, low, close, volume, timeperiod=14)
    df['MINUS_DI'] = talib.MINUS_DI(high, low, close, timeperiod=14)
    df['MINUS_DM'] = talib.MINUS_DM(high, low, timeperiod=14)
    df['MOM'] = talib.MOM(close, timeperiod=10)
    df['PLUS_DI'] = talib.PLUS_DI(high, low, close, timeperiod=14)
    df['PLUS_DM'] = talib.PLUS_DM(high, low, timeperiod=14)
    df['RSI'] = talib.RSI(close, timeperiod=14)
    df['STOCH_slowk'], df['STOCH_slowd'] = talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df['STOCHF_fastk'], df['STOCHF_fastd'] = talib.STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)
    df['STOCHRSI_fastk'], df['STOCHRSI_fastd'] = talib.STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    df['TRIX'] = talib.TRIX(close, timeperiod=30)
    df['ULTOSC'] = talib.ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    df['WILLR'] = talib.WILLR(high, low, close, timeperiod=14)

    df['AD'] = talib.AD(high, low, close, volume)
    df['ADOSC'] = talib.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
    df['OBV'] = talib.OBV(close, volume)

    df['ATR'] = talib.ATR(high, low, close, timeperiod=14)
    df['NATR'] = talib.NATR(high, low, close, timeperiod=14)
    df['TRANGE'] = talib.TRANGE(high, low, close)

    df['HT_DCPERIOD'] = talib.HT_DCPERIOD(close)
    df['HT_DCPHASE'] = talib.HT_DCPHASE(close)
    df['HT_PHASOR_inphase'], df['HT_PHASOR_quadrature'] = talib.HT_PHASOR(close)
    df['HT_SINE_sine'], df['HT_SINE_leadsine'] = talib.HT_SINE(close)
    df['HT_TRENDMODE'] = talib.HT_TRENDMODE(close)

    df['BETA'] = talib.BETA(high, low, timeperiod=5)
    df['CORREL'] = talib.CORREL(high, low, timeperiod=30)
    df['LINEARREG'] = talib.LINEARREG(close, timeperiod=14) - close
    df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(close, timeperiod=14)
    df['LINEARREG_INTERCEPT'] = talib.LINEARREG_INTERCEPT(close, timeperiod=14) - close
    df['LINEARREG_SLOPE'] = talib.LINEARREG_SLOPE(close, timeperiod=14)
    df['STDDEV'] = talib.STDDEV(close, timeperiod=5, nbdev=1)

    return df

---

# Main 

主処理

In [40]:
df_btc = pd.read_csv("../data/nb002/btc_jpy_15min.csv")
df_btc.columns = ["time", "op", "hi", "lo", "cl", "volume"]

In [41]:
# 12月データをテストとして切り出し
df_btc_train = df_btc[pd.to_datetime(df_btc["time"]) < pd.to_datetime("2021-12-01")]
df_btc_test = df_btc[pd.to_datetime(df_btc["time"]) >= pd.to_datetime("2021-12-01")]

In [42]:
df_btc_train, df_btc_test

(                     time       op       hi       lo       cl     volume
 0     2021-10-27 13:15:00  6720546  6738258  6710000  6735988  20.589480
 1     2021-10-27 13:30:00  6738258  6739563  6708000  6710000  14.918648
 2     2021-10-27 13:45:00  6711247  6722993  6700000  6722993  22.997577
 3     2021-10-27 14:00:00  6720612  6736192  6704518  6710000  17.630379
 4     2021-10-27 14:15:00  6713667  6723157  6704320  6704320  16.902241
 ...                   ...      ...      ...      ...      ...        ...
 3302  2021-11-30 22:45:00  6504166  6506439  6485316  6500000   6.466320
 3303  2021-11-30 23:00:00  6496913  6500930  6479772  6491062   5.330892
 3304  2021-11-30 23:15:00  6488943  6503746  6462535  6470978  23.821248
 3305  2021-11-30 23:30:00  6475942  6501111  6472106  6493391  16.240694
 3306  2021-11-30 23:45:00  6493392  6495810  6466255  6479057  21.825394
 
 [3307 rows x 6 columns],
                      time       op       hi       lo       cl     volume
 3307  202

In [43]:
df_feat = calc_features(df_btc_train)

<ipython-input-39-549ecda6586e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BBANDS_upperband'], df['BBANDS_middleband'], df['BBANDS_lowerband'] = talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
<ipython-input-39-549ecda6586e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BBANDS_upperband'] -= hilo
<ipython-input-39-549ecda6586e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [44]:
df_feat

,time,op,hi,lo,cl,volume,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,DEMA,...,HT_SINE_sine,HT_SINE_leadsine,HT_TRENDMODE,BETA,CORREL,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,STDDEV
0,2021-10-27 13:15:00,6720546,6738258,6710000,6735988,20.589480,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-27 13:30:00,6738258,6739563,6708000,6710000,14.918648,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-10-27 13:45:00,6711247,6722993,6700000,6722993,22.997577,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-10-27 14:00:00,6720612,6736192,6704518,6710000,17.630379,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-10-27 14:15:00,6713667,6723157,6704320,6704320,16.902241,25806.896487,2921.7,-19963.496487,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,11442.598244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3302,2021-11-30 22:45:00,6504166,6506439,6485316,6500000,6.466320,26528.407742,-14235.5,-54999.407742,4581.624408,...,0.282671,0.878147,0,1.057645,0.881033,-22127.771429,-89.983638,23394.342857,-3501.701099,20381.953871
3303,2021-11-30 23:00:00,6496913,6500930,6479772,6491062,5.330892,24961.231702,-445.2,-25851.631702,8629.555155,...,0.553273,0.980243,0,0.497771,0.845154,-8883.857143,-89.974116,19892.000000,-2213.527473,12703.215851
3304,2021-11-30 23:15:00,6488943,6503746,6462535,6470978,23.821248,31670.574594,6999.9,-17670.774594,12005.267472,...,0.668241,0.998566,0,0.648940,0.766211,7527.771429,-89.974800,37085.514286,-2273.672527,12335.337297
3305,2021-11-30 23:30:00,6475942,6501111,6472106,6493391,16.240694,29244.848269,5626.1,-17992.648269,7879.444881,...,0.736972,0.999068,1,0.473873,0.824342,-10397.685714,-89.953844,5739.971429,-1241.358242,11809.374134


In [45]:
df_feat.dropna()

,time,op,hi,lo,cl,volume,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,DEMA,...,HT_SINE_sine,HT_SINE_leadsine,HT_TRENDMODE,BETA,CORREL,LINEARREG,LINEARREG_ANGLE,LINEARREG_INTERCEPT,LINEARREG_SLOPE,STDDEV
88,2021-10-28 11:15:00,6941093,6945820,6923289,6923289,8.492828,21399.589381,4130.9,-13137.789381,-34655.688807,...,0.380098,-0.385266,1,-0.080960,0.987324,61453.314286,89.996419,-146551.600000,16000.378022,8634.344691
89,2021-10-28 11:30:00,6923289,6938000,6911450,6926136,20.001185,31928.996513,11849.4,-8230.196513,-16774.700420,...,0.252738,-0.505438,1,0.721980,0.987655,46264.914286,89.995126,-106557.342857,11755.558242,10039.798257
90,2021-10-28 11:45:00,6928000,6933795,6918711,6920519,9.707317,25994.204748,5205.6,-15583.004748,-11882.300279,...,0.107700,-0.626839,1,0.779778,0.987466,35815.885714,89.992144,-58996.600000,7293.268132,10394.302374
91,2021-10-28 12:00:00,6920519,6937000,6920232,6933481,20.071957,14304.971733,69.0,-14166.971733,-6982.459780,...,-0.036076,-0.732156,1,0.393791,0.987338,12454.971429,89.986154,-41339.828571,4138.061538,7117.985867
92,2021-10-28 12:15:00,6933478,6947765,6924658,6941647,19.353866,8107.025226,-7197.1,-22501.225226,-7153.028772,...,-0.192366,-0.829924,1,0.420704,0.987681,1352.771429,89.979277,-34590.057143,2764.832967,7652.062613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3302,2021-11-30 22:45:00,6504166,6506439,6485316,6500000,6.466320,26528.407742,-14235.5,-54999.407742,4581.624408,...,0.282671,0.878147,0,1.057645,0.881033,-22127.771429,-89.983638,23394.342857,-3501.701099,20381.953871
3303,2021-11-30 23:00:00,6496913,6500930,6479772,6491062,5.330892,24961.231702,-445.2,-25851.631702,8629.555155,...,0.553273,0.980243,0,0.497771,0.845154,-8883.857143,-89.974116,19892.000000,-2213.527473,12703.215851
3304,2021-11-30 23:15:00,6488943,6503746,6462535,6470978,23.821248,31670.574594,6999.9,-17670.774594,12005.267472,...,0.668241,0.998566,0,0.648940,0.766211,7527.771429,-89.974800,37085.514286,-2273.672527,12335.337297
3305,2021-11-30 23:30:00,6475942,6501111,6472106,6493391,16.240694,29244.848269,5626.1,-17992.648269,7879.444881,...,0.736972,0.999068,1,0.473873,0.824342,-10397.685714,-89.953844,5739.971429,-1241.358242,11809.374134


# 戦略 - ストラテジーを考える

## シンプル

- 上げトレンド→買い

- 下げトレンド→売り